In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [8]:
DB_USER = "rishika" # put db user here
DB_PASSWORD = "21062004"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "vestige"

ENGINE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(ENGINE_URL)

In [10]:
enrolment_df = pd.read_sql("SELECT date, state, district FROM raw.enrolment", engine)

demographic_df = pd.read_sql("SELECT date, state, district FROM raw.demographic", engine)

biometric_df = pd.read_sql("SELECT date, state, district FROM raw.biometric", engine)

In [11]:
print("Row counts")
print("Enrolment   :", len(enrolment_df))
print("Demographic :", len(demographic_df))
print("Biometric   :", len(biometric_df))

Row counts
Enrolment   : 1006029
Demographic : 2071700
Biometric   : 1861108


In [12]:
def null_report(df, name):
    print(f"\n{name}")
    print(df[["date", "state", "district"]].isna().mean())

null_report(enrolment_df, "Enrolment")
null_report(demographic_df, "Demographic")
null_report(biometric_df, "Biometric")


Enrolment
date        0.0
state       0.0
district    0.0
dtype: float64

Demographic
date        0.0
state       0.0
district    0.0
dtype: float64

Biometric
date        0.0
state       0.0
district    0.0
dtype: float64


In [13]:
def unique_states(df, name):
    states = sorted(df["state"].dropna().unique())
    print(f"\n{name} unique states ({len(states)}):")
    return states

enrolment_states = unique_states(enrolment_df, "Enrolment")
demographic_states = unique_states(demographic_df, "Demographic")
biometric_states = unique_states(biometric_df, "Biometric")


Enrolment unique states (55):

Demographic unique states (65):

Biometric unique states (57):


In [14]:
print("\nStates only in enrolment:", set(enrolment_states) - set(demographic_states) - set(biometric_states))

print("\nStates only in demographic:", set(demographic_states) - set(enrolment_states) - set(biometric_states))

print("\nStates only in biometric:", set(biometric_states) - set(enrolment_states) - set(demographic_states))


States only in enrolment: {'The Dadra And Nagar Haveli And Daman And Diu', 'Jammu And Kashmir'}

States only in demographic: {'Darbhanga', 'BALANAGAR', 'Nagpur', 'Madanapalle', 'West Bengli', 'Jaipur', 'Raja Annamalai Puram', 'Puttenahalli'}

States only in biometric: {'Tamilnadu'}


In [15]:
for df in (enrolment_df, demographic_df, biometric_df):
    df["date"] = pd.to_datetime(df["date"], errors="raise")
    df["state"] = df["state"].astype("string")
    df["district"] = df["district"].astype("string")

In [20]:
enrolment_df.to_parquet("../sql/data/data_cache/enrolment_raw.parquet", engine="fastparquet")
demographic_df.to_parquet("../sql/data/data_cache/demographic_raw.parquet", engine="fastparquet")
biometric_df.to_parquet("../sql/data/data_cache/biometric_raw.parquet", engine="fastparquet")